### 1. Transformer Architecture



attention 계산식을 생각하며 빈칸을 채워봅시다!!

In [2]:
import math
import random
from typing import Tuple

import torch
import torch.nn as nn


# -------------------------
# A. Scaled Dot-Product Attention
# -------------------------
class ScaledDotProductAttention(nn.Module):
    def __init__(self, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

    def forward(self, Q, K, V, mask=None):
        """
        Q,K,V: (batch, heads, seq_len, d_k)
        mask:  (batch, 1 or heads, seq_len, seq_len)
        """
        d_k = Q.size(-1)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)  # (B,H,L,L)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        output = torch.matmul(attn, V)  # (B,H,L,d_k)
        return output, attn


## 문제 1) 아래 코드를 살펴보고, 단순 Attention 대신 Multi-Head Attention을 사용하는 이유를 설명하시오.

**답안)** 단일 Scaled Dot-Product Attention은 하나의 시각(하나의 표현 공간) 에서만 Query와 Key의 유사도를 계산한다. 하지만 언어 데이터는 단어의 의미, 문맥, 위치 정보 등이 매우 다양하기 때문에, 단일 관점만으로는 충분히 복잡한 관계를 포착하기 어렵다.

Multi-Head Attention은 입력을 여러 개의 헤드(head)로 나누어 여러 다른 표현 공간에서 병렬적으로 서로 다른 관점을 학습하게 만든다.


*   어떤 헤드는 문법적 관계(주어–동사 일치)를,
*   어떤 헤드는 의미적 유사성(동의어 관계)을,
*   또 다른 헤드는 문맥적 거리(앞뒤 단어 간 의존성)를 포착할 수 있다.


이렇게 다양한 관계를 동시에 학습한 뒤, 다시 결합하면 풍부한 표현력을 가진 결과를 얻을 수 있다.
→ 따라서 Multi-Head Attention은 단순 Attention보다 더 복잡한 패턴과 장기 의존성을 효과적으로 학습할 수 있는 장점이 있다.

## 문제 2) Positional Encoding의 기능에 대해 설명하시오.
**답안)** Transformer는 RNN과 달리 순서를 차례대로 처리하지 않고, 모든 토큰을 동시에 병렬 처리한다. 따라서 모델 자체에는 단어의 위치 정보가 없다. 이를 보완하기 위해 Positional Encoding을 사용한다.

*   각 단어 벡터에 위치 정보를 담은 벡터를 더해 준다.
*   이렇게 하면 모델이 단어의 상대적·절대적 위치를 학습할 수 있어, 순서 정보가 없는 Self-Attention 구조에서도 문맥적 순서를 반영할 수 있다.

즉, Positional Encoding은 "문장의 단어 순서 정보"를 Transformer에 주입하는 핵심 역할을 한다.

In [3]:
# -------------------------
# B. Multi-Head Attention
# -------------------------
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_k = d_model // num_heads
        self.num_heads = num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.layernorm = nn.LayerNorm(d_model)

        self.attn = ScaledDotProductAttention(dropout=dropout)

    def _split_heads(self, x):
        B, L, D = x.shape
        x = x.view(B, L, self.num_heads, self.d_k).transpose(1, 2)
        return x

    def _combine_heads(self, x):
        B, H, L, d_k = x.shape
        x = x.transpose(1, 2).contiguous().view(B, L, H * d_k)
        return x

    def forward(self, x, kv=None, mask=None):
        residual = x
        if kv is None:
            kv = x
        Q = self._split_heads(self.W_q(x))
        K = self._split_heads(self.W_k(kv))
        V = self._split_heads(self.W_v(kv))
        ctx, _ = self.attn(Q, K, V, mask=mask)          # (B,H,L_q,d_k)
        out = self._combine_heads(ctx)                  # (B,L_q,D)
        out = self.dropout(self.W_o(out))
        return self.layernorm(out + residual)

# -------------------------
# C. Positional Encoding (sin/cos)
# -------------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-(math.log(10000.0) / d_model)))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # (1,L,D)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :].to(x.dtype)
        return self.dropout(x)

# -------------------------
# D. Position-wise FFN
# -------------------------
class PositionwiseFFN(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout),
        )
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        residual = x
        x = self.net(x)
        return self.norm(x + residual)

Transformer의 인코더와 디코더 레이어 구조를 생각하며 빈칸을 채워봅시다!!

In [4]:
# -------------------------
# E. Encoder
# -------------------------
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ffn = PositionwiseFFN(d_model, d_ff, dropout)

    def forward(self, x, src_mask=None):
        x = self.self_attn(x, kv=None, mask=src_mask)
        x = self.ffn(x)
        return x

class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, num_heads, d_ff, dropout=0.1, max_len=5000):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
        self.posenc = PositionalEncoding(d_model, max_len, dropout)
        self.layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(N)])

    def forward(self, src, src_mask=None):
        x = self.embed(src) * math.sqrt(d_model := self.embed.embedding_dim)
        x = self.posenc(x)
        for layer in self.layers:
            x = layer(x, src_mask=src_mask)
        return x

# -------------------------
# F. Decoder
# -------------------------
def generate_subsequent_mask(sz: int):
    mask = torch.tril(torch.ones(sz, sz)).bool()
    return mask.unsqueeze(0).unsqueeze(0)

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.cross_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ffn = PositionwiseFFN(d_model, d_ff, dropout)

    def forward(self, x, enc_out, tgt_mask=None, memory_mask=None):
        x = self.self_attn(x, kv=None, mask=tgt_mask)
        x = self.cross_attn(x, kv=enc_out, mask=memory_mask)
        x = self.ffn(x)
        return x

class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, num_heads, d_ff, dropout=0.1, max_len=5000):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
        self.posenc = PositionalEncoding(d_model, max_len, dropout)
        self.layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(N)])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, tgt, enc_out, tgt_mask=None, memory_mask=None):
        x = self.embed(tgt) * math.sqrt(d_model := self.embed.embedding_dim)
        x = self.posenc(x)
        for layer in self.layers:
            x = layer(x, enc_out, tgt_mask=tgt_mask, memory_mask=memory_mask)
        return self.norm(x)

# -------------------------
# G. 전체 Transformer + 마스크
# -------------------------
class Transformer(nn.Module):
    def __init__(self, src_vocab, tgt_vocab, d_model=256, N=4, heads=4, d_ff=1024, dropout=0.1, max_len=512):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads, d_ff, dropout, max_len)
        self.decoder = Decoder(tgt_vocab, d_model, N, heads, d_ff, dropout, max_len)
        self.generator = nn.Linear(d_model, tgt_vocab)

    def make_src_mask(self, src):
        return (src != PAD).unsqueeze(1).unsqueeze(1)  # (B,1,1,Ls)

    def make_tgt_mask(self, tgt):
        B, L = tgt.shape
        pad = (tgt != PAD).unsqueeze(1).unsqueeze(1)   # (B,1,1,Lt)
        causal = generate_subsequent_mask(L).to(tgt.device)  # (1,1,Lt,Lt)
        return pad & causal

    def forward(self, src, tgt):
        src_mask = self.make_src_mask(src)
        tgt_mask = self.make_tgt_mask(tgt)
        memory = self.encoder(src, src_mask=src_mask)
        out = self.decoder(tgt, memory, tgt_mask=tgt_mask, memory_mask=src_mask)
        logits = self.generator(out)
        return logits



### 2. Self-Supervised Learning

#### 문제 1) Autoencoding Language Model
아래 세 문장에서 BERT가 [MASK] 위치에 대해 예측한 1순위 토큰이 문맥상 적절한지 평가하세요.

적절하다면, 왜 해당 토큰이 자연스럽다고 볼 수 있는지 근거를 제시하세요.

적절하지 않다면, 그 이유가 문맥 이해 부족 때문인지, 아니면 훈련 데이터 분포(자주 등장하는 표현) 때문인지 분석해 보세요.

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

mlm_name = "distilbert-base-uncased"  # 경량 BERT
tok = AutoTokenizer.from_pretrained(mlm_name)
model = AutoModelForMaskedLM.from_pretrained(mlm_name)
model.eval()

def topk_mask_fill(text, k=5):
    inputs = tok(text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    mask_idx = (inputs.input_ids[0] == tok.mask_token_id).nonzero(as_tuple=True)[0].item()
    probs = torch.softmax(logits[0, mask_idx], dim=-1)
    topk_ids = torch.topk(probs, k=k).indices.tolist()
    return [(tok.decode([i]), float(probs[i])) for i in topk_ids]

sentences = [
    "I'm wondering if I should eat [MASK] for lunch today.",
    "I decided to go to the [MASK] with my friends this weekend.",
    "It started to rain and I remembered I left my umbrella at [MASK]."
]

for s in sentences:
    print("\nInput:", s)
    preds = topk_mask_fill(s, k=5) # top-k 자유롭게 수정 가능
    for t,p in preds:
        print(f"  - {t:15s}  p={p:.4f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]


Input: I'm wondering if I should eat [MASK] for lunch today.
  - something        p=0.0690
  - here             p=0.0688
  - breakfast        p=0.0414
  - dinner           p=0.0405
  - pizza            p=0.0374

Input: I decided to go to the [MASK] with my friends this weekend.
  - beach            p=0.1086
  - movies           p=0.0727
  - gym              p=0.0478
  - mall             p=0.0325
  - zoo              p=0.0305

Input: It started to rain and I remembered I left my umbrella at [MASK].
  - home             p=0.0831
  - night            p=0.0602
  - school           p=0.0366
  - dawn             p=0.0308
  - lunch            p=0.0271


## **답안)**
입력 문장: 오늘 점심으로 [MASK]를 먹어야 할지 고민 중이야.<br>
예측 단어:


*   something → 무언가 (p=0.0690)
*   here → 여기서 (p=0.0688)
*   breakfast → 아침식사 (p=0.0414)
*   dinner → 저녁식사 (p=0.0405)
*   pizza → 피자 (p=0.0374)

 예측한 1순위 토큰은 something으로 **적절하다**. 영어에서 "eat something for lunch"는 매우 자연스러운 표현이기 때문이다.

---
입력 문장: 이번 주말에 친구들과 함께 [MASK]에 가기로 했어 <br>
예측 단어:


*   beach → 해변 (p=0.1086)
*   movies → 영화관 (p=0.0727)
*   gym → 체육관(헬스장) (p=0.0478)
*   mall → 쇼핑몰 (p=0.0325)
*   zoo → 동물원 (p=0.0305)

예측한 1순위 토큰은 beach로 **적절하다**. 친구들과 주말에 가는 장소로 "beach"는 매우 자연스럽다. movies, mall, gym 모두 문맥상 타당하지만 훈련 데이터에서 자주 등장하는 레저 활동 패턴 때문에 beach가 선택된 것으로 보인다.

---
입력 문장: 비가 오기 시작했을 때, 내가 우산을 [MASK]에 두고 온 것을 떠올렸어 <br>
예측 단어:


*   home → 집 (p=0.0831)
*   night → 밤 (p=0.0602)
*   school → 학교 (p=0.0366)
*   dawn → 새벽 (p=0.0308)
*   lunch → 점심 (p=0.0271)


예측한 1순위 토큰은 home으로 **적절하다** 비가 오는데 우산을 두고 온 장소로 가장 자연스러운 건 "집"이다. 다른 후보인 school도 맞을 수 있지만, 일반적으로는 home이 가장 흔하다.


### 3. Prompt Engineering

아래는 동일한 질문에 대해 Baseline Prompt와 Engineered Prompt를 사용했을 때의 모델 답변이다.
두 결과를 비교하고, 왜 프롬프트 엔지니어링(prompt engineering)이 중요한지 서술하시오.

또한, 프롬프트 엔지니어링 기법에 대해 설명하시오.

**답안)** Baseline Prompt에서는 "Convert March 5th, 2024 to YYYY-MM-DD format"이라고 규칙을 제시했음에도 불구하고 모델은 "5th, 2024"라는 부정확한 출력을 내놓았다. 이는 규칙이 사람에게는 명확해 보이지만, 모델 입장에서는 해석의 여지가 남아 있어 단순히 원문을 부분적으로 재생산하는 결과가 발생한 것이다.

반면 Engineered Prompt는 "You are a date parser"라는 역할을 지정하고, 연도는 네 자리, 월과 일은 두 자리 숫자로 표기한다는 구체적 규칙을 세분화하여 제시했으며, 출력 형식도 "날짜만 출력하라"는 제약조건을 포함했다. 그 결과 모델은 더 정확하고 의도에 가까운 출력을 생성할 수 있었다. 이는 동일한 모델이라도 프롬프트의 설계 방식에 따라 성능과 결과의 품질이 크게 달라진다는 점을 보여준다. 따라서 프롬프트 엔지니어링은 모델 활용에서 핵심적인 역할을 한다고 할 수 있다.


---
프롬프트 엔지니어링 기법에는 여러 가지가 있다. 먼저, 질**문에 세부사항을 포함하기**는 모호한 지시 대신 구체적인 요구 사항을 적시하여 모델이 원하는 방향으로 답변하도록 돕는다. **페르소나 부여하기**는 모델에게 특정 역할이나 인격을 부여하여 답변의 톤과 일관성을 높이는 방법이다. **구분자 활용하기**는 따옴표나 태그와 같은 구분 기호를 사용하여 입력 요소를 명확히 분리함으로써 모델이 혼동하지 않게 한다. **작업 단계를 명시하기**는 복잡한 문제를 단계별로 해결하도록 유도하는 것으로, 이는 곧 **Chain-of-Thought Prompting**과 연결된다. 또한 **예시(Few-shot) 제공하기**는 입력과 출력의 구체적 사례를 포함해 원하는 패턴을 학습하게 만드는 방식이다. 마지막으로, **출력 형식이나 길이를 지정하기**는 JSON, 표, 글자 수 제한 등 명확한 제약 조건을 통해 불필요한 정보를 줄이고 원하는 결과를 안정적으로 얻는 방법이다.

이러한 기법들을 종합하면, Role Prompting을 통해 역할을 지정하고, Instruction Prompting으로 작업을 구체적으로 지시하며, Constraint Prompting으로 출력 범위를 제한하고, Example-based Prompting으로 예시를 제시하며, Chain-of-Thought Prompting으로 단계적 추론을 유도하는 방식으로 요약할 수 있다.


In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# -----------------------------
# 1) 모델 로드
# -----------------------------
model_id = "google/flan-t5-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)

def generate(prompt, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# -----------------------------
# 2) 예제 프롬프트들 (Baseline vs Engineered)
# -----------------------------
prompts = {
    "convert date": {
        "baseline": '''Convert March 5th, 2024 to YYYY-MM-DD format ''',

        "engineered": '''You are a date parser.
Task: Convert the input into exactly YYYY-MM-DD format (4-digit year, 2-digit month, 2-digit day).
Rules:
- Output ONLY the date in that format.
- No extra text or explanation.
Input: "March 5th, 2024"
Output:'''
    }

    }


# -----------------------------
# 3) 실행 및 비교 출력
# -----------------------------
for task, variants in prompts.items():
    print("="*80)
    print(f"📝 Task: {task}")

    for kind, prompt in variants.items():
        output = generate(prompt)
        print(f"\n--- {kind.upper()} Prompt ---")
        print(prompt)
        print(f"\n👉 Model Output:\n{output}\n")


📝 Task: convert date

--- BASELINE Prompt ---
Convert March 5th, 2024 to YYYY-MM-DD format 

👉 Model Output:
5th, 2024


--- ENGINEERED Prompt ---
You are a date parser.
Task: Convert the input into exactly YYYY-MM-DD format (4-digit year, 2-digit month, 2-digit day).
Rules:
- Output ONLY the date in that format.
- No extra text or explanation.
Input: "March 5th, 2024"
Output:

👉 Model Output:
"5/05/2024"



### 4.RAG


In [7]:
!pip install -qU langchain langchain_community sentence-transformers faiss-cpu transformers accelerate langchain-core langchain-upstage bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#### 문제1) 아래 코드를 참고하여 RAG 프로세스를 서술해주세요

**답안)** <br>

**1. 문서 업로드 및 로드**


*   사용자가 Colab에 문서를 업로드하면 TextLoader를 통해 문서 내용을 불러온다.
*   이 단계에서 모델이 답변을 생성할 때 활용할 원천 지식이 확보된다.


**2. 문서 분할 (Chunking)**


*   긴 문서는 직접 임베딩하기 어렵기 때문에 RecursiveCharacterTextSplitter를 이용해 일정 길이(여기서는 1000자, 50자 겹침)로 나눈다.
*   이렇게 하면 검색 시 더 세밀하게 관련 정보를 찾을 수 있다.


**3. 임베딩(Embedding) 생성 및 벡터 저장소 구축**

*   HuggingFaceEmbeddings 모델(all-MiniLM-L6-v2)을 사용해 각 청크를 벡터로 변환한다.
*   변환된 벡터들은 FAISS(Vector Database)에 저장되어, 이후 질의와의 유사도를 계산하는 기반이 된다.


**4. Retriever 생성 (검색기)**

*   FAISS 저장소에서 질의와 가장 유사한 청크를 찾아오는 모듈.
*   검색 시 상위 k개의 문서 청크를 반환한다.


**5. 프롬프트 템플릿 구성**

*   검색된 문맥과 사용자의 질문을 하나의 입력으로 묶는다.


**6. RAG 체인 구성**

*   RetrievalQA를 이용해 LLM(Solar-Pro2)과 검색기(Retriever)를 연결한다.
*   질의가 들어오면 → 관련 문서 검색 → 문맥과 질문을 합쳐 LLM에 전달 → 답변 생성.


**7. 대화형 질의 응답**

*   사용자가 질문을 입력하면, 시스템은 검색된 문맥 기반으로 답변을 반환한다.
*   관련 내용이 없으면 "관련 내용을 찾을 수 없습니다."라고 출력한다.

**8. 벡터 저장소 저장**

*   생성된 FAISS 인덱스를 로컬 디렉토리에 저장해두어, 이후 재활용 가능하도록 한다.




#### 문제2) sample.md 를 업로드해서 아래 샘플 질문들을 입력해 결과를 출력해보세요.

1. "인공지능의 역사에서 튜링 테스트란 무엇인가요?"
2. "딥러닝 혁명은 언제 시작되었나요?"
3. "데이터 분석에 대해 설명하세요"


In [ ]:
import os, tempfile
from google.colab import files
from langchain_upstage import ChatUpstage
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

# 파일 업로드 함수
def upload_file():
    print("문서 파일을 업로드해주세요.")
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]
    print(f"업로드 완료: {file_path}")
    return file_path

# -----------------------------
# 1) Solar-Pro2 LLM 로드
# -----------------------------
chat = ChatUpstage(
    api_key="up_qwEgtTW1CNtpfl7ZeIb9MUmsWHIBp",
    model="solar-pro2"
)

# -----------------------------
# 2) Colab RAG 시스템 정의
# -----------------------------
def colab_rag_system(file_path):
    # 1. 문서 로드
    loader = TextLoader(file_path)
    documents = loader.load()
    print(f"문서 로딩 완료: {len(documents)} 개의 문서")

    # 2. 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=50,
        length_function=len
    )
    texts = text_splitter.split_documents(documents)
    print(f"문서 분할 완료: {len(texts)} 개의 청크")

    # 3. 임베딩 + 벡터저장소
    print("임베딩 모델 로딩 중...")
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    print("벡터 저장소 구축 중...")
    vectorstore = FAISS.from_documents(texts, embeddings)

    # 4. 검색기 생성
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

    # 5. 프롬프트 템플릿
    prompt_template = """
    다음 정보를 바탕으로 질문에 답해주세요.
    만약 관련 내용이 없다면 "관련 내용을 찾을 수 없습니다."라고 답해주세요.

    {context}

    질문: {question}
    답변:
    """
    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    # 6. RAG 파이프라인 구축
    print("RAG 파이프라인 구축 중...")
    qa_chain = RetrievalQA.from_chain_type(
        llm=chat,                    # 여기서 Solar-Pro2 사용
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )

    print("RAG 시스템 준비 완료!")

    # 7. 대화형 질의
    while True:
        query = input("\n질문을 입력하세요 (종료하려면 'q' 입력): ")
        if query.lower() == 'q':
            break

        result = qa_chain.invoke({"query": query})

        print("\n답변:", result["result"])


    # 8. 벡터 저장소 저장
    with tempfile.TemporaryDirectory() as temp_dir:
        index_path = os.path.join(temp_dir, "faiss_index")
        vectorstore.save_local(index_path)
        print(f"\n인덱스를 '{index_path}'에 저장했습니다.")

if __name__ == "__main__":
    file_path = upload_file()
    colab_rag_system(file_path)

문서 파일을 업로드해주세요.


Saving sample-document (1).md to sample-document (1) (1).md
업로드 완료: sample-document (1) (1).md
문서 로딩 완료: 1 개의 문서
문서 분할 완료: 2 개의 청크
임베딩 모델 로딩 중...
벡터 저장소 구축 중...
RAG 파이프라인 구축 중...
RAG 시스템 준비 완료!

질문을 입력하세요 (종료하려면 'q' 입력): "인공지능의 역사에서 튜링 테스트란 무엇인가요?

답변: 튜링 테스트는 앨런 튜링이 1950년 논문 "Computing Machinery and Intelligence"에서 제안한 개념으로, **기계가 인간과 구별할 수 없는 지능적인 행동을 보일 수 있는지**를 평가하는 방법입니다. 이 테스트는 인간 평가자가 텍스트 기반 대화(예: 질문-답변)를 통해 기계와 인간을 구분하지 못할 경우, 해당 기계가 "지능적"이라고 판단하는 기준을 제시합니다.  

### 주요 내용:
- **목적**: 기계의 사고 능력을 간접적으로 측정하는 도구로 활용.
- **방법**: 평가자는 인간과 기계 중 어느 쪽과 대화하는지 모른 상태에서 대화를 진행하며, 기계를 인간으로 오인한 비율이 일정 수준 이상일 경우 테스트 통과로 간주.
- **의의**: AI의 철학적·기술적 기준을 제시했으나, 비판도 존재합니다(예: 감정 또는 창의성 평가 불가).  

관련 내용은 "인공지능의 역사와 발전" 섹션의 **초기 인공지능 (1940-1950년대)** 부분에서 확인할 수 있습니다.

질문을 입력하세요 (종료하려면 'q' 입력): 딥러닝 혁명은 언제 시작되었나요?

답변: 딥러닝 혁명은 **2010년대**에 시작되었습니다. 특히 **2012년 ImageNet 대회에서 알렉스넷(AlexNet)의 성공**이 딥러닝의 잠재력을 입증하며 본격적인 발전의 전환점이 되었습니다. 이는 컴퓨팅 파워의 증가, 빅데이터의 가용성, 알고리즘 개선 등과 함께 컴퓨터 비전, 음성 인식, 자연어 처리 등 다양한 분야에서 혁신을 이끌었습